# Data Story

In [91]:
import numpy as np
import pandas as pd

import re

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [127]:
px.defaults.template = "simple-white"

In [71]:
xlsx = pd.ExcelFile('data_collectionklimadaten_swiss_open_data.xlsx')
swiss_sunshine = pd.read_excel(xlsx, 'Sonnenscheindauer' ,
                                   header=5, index_col=0 ).dropna().iloc[1:]

swiss_precipitation = pd.read_excel(xlsx, 'Jahresniederschlag' ,
                                         header=5, index_col=0 ).dropna().iloc[1:]

swiss_temp = pd.read_excel(xlsx, 'Jahrestemperatur' ,
                                header=5, index_col=0 ).dropna().iloc[1:]

swiss_snow = pd.read_excel(xlsx, 'Neuschnee' ,
                           header=5, index_col=0 ).dropna().iloc[1:]


co2_data = pd.read_csv('co2.csv', index_col=1).drop(['Unnamed: 0'], axis=1)
co2_data.index = pd.to_datetime(co2_data.index)

temp_data = pd.read_csv('temperature_global.csv', index_col=1).drop(['Unnamed: 0'], axis=1)
temp_data.index = pd.to_datetime(temp_data.index)

In [72]:
def swiss_places_column_titles(data):
    """Replaces the Header of the swiss data table with only alphabetic chars."""
    new_cols = []
    
    for i in data.columns:
        result = re.sub("[1,2) ]", '', i)
        new_cols.append(result)
    data.columns = new_cols
    
    return data  

In [73]:
swiss_sunshine = swiss_places_column_titles(swiss_sunshine)
swiss_precipitation = swiss_places_column_titles(swiss_precipitation)
swiss_snow = swiss_places_column_titles(swiss_snow)
swiss_temp = swiss_places_column_titles(swiss_temp)

Jan geht jeden Winter  zu seiner Grossmutter in die Skiferien nach Davos. Seine Grossmutter lebt bereits ihr ganzes Leben im Davoser-Dorf. Immer wieder tönt sie an, es gäbe von Jahr zu Jahr immer weniger Schnee im Dorf und auch im Skigebiet. Jan ist sehr faktenorientiert und möchte sich dies einmal genauer ansehen. Er recherchiert...

Jan findet eine Infografik die jährliche gemessene Schneehöhe in Davos-Klosters.

In [130]:
def lineplot_davos_snow(data):
    
    data = data['DavosWSL']
    new_val = np.where(data == '...' ,None , data*1)
    data = pd.DataFrame(new_val, index=data.index).dropna().astype('int')

    data = data - data.iloc[0]
    data.loc[:,'color'] = np.where(data >= 0, 1, 0)
    data.columns = ['snow', 'color']
        
    fig = px.bar(x=data.index, y=data['snow'], color=data['color'] ,
                 title='Amount of Snow in cm',
                 labels={'x':'Years', 'y':'Amount of Snow (cm)'}, 
                 color_continuous_scale=['red', 'blue'], 
                 template='simple_white')
    
    fig.update(layout_coloraxis_showscale=False)
    fig.show()
    
    
lineplot_davos_snow(swiss_snow)

In [10]:
def lineplot_swiss_temp(data):
    for column in data.columns:
        data[column] = data[column] - data[column].iloc[0]
    data = pd.melt(data.reset_index(), id_vars='index')
    data.columns = ['year', 'location', 'temperature']
    data = data[data['location'] == 'Davos']
    
    fig = px.line(x=data['year'], y=data['temperature'], color=data['location'], 
                  title='Temperature Change from initial Temperature in K',
                 labels={'x':'Years', 'y':'Temperature Difference in K'})
    fig.show()
    
    
lineplot_swiss_temp(swiss_temp)